Convert audio file to .wav format if necessary. Change the filename parameter of the _AudioSegment.from_mp3()_ segment to march the filename of your input audio file


In [26]:
from pydub import AudioSegment

# load the mp3 file as an AudioSegment object
audio = AudioSegment.from_mp3("Sanchez - Mama (African Pride).mp3")

# export the audio as a wav file
audio.export("audio.wav", format="wav")

<_io.BufferedRandom name='audio.wav'>

Generate a fingerprint from .wav audio segment.

Requirements:

- MySQL database, where Dejavu can store fingerprints. Run this code to create a new MySQL db:
  <strong>mysql -u root -p</strong>

Edit the database configurations for your storage instance


In [ ]:
import numpy
print(numpy.__version__)
print(numpy.__path__)

In [ ]:
import json
from dejavu import Dejavu

### mysql  Ver 8.0.32-0ubuntu0.20.04.2 for Linux on x86_64 ((Ubuntu))
config = {
    "database": {
        "host": "localhost",
        "user": "dejavu",
        "password": "dejavu9466",
        "database": "dejavu",
    },
    "database_type": "mysql",
    "fingerprint_limit": 20,
}
djv = Dejavu(config)

# any subsequent calls to fingerprint_file or fingerprint_directory 
# will fingerprint and add those songs to the database ehose connection
# has been defined in the config dictionary above. 
# It's meant to simulate a system where as new songs are released, 
# they are fingerprinted and added to the database seemlessly without 
# stopping the system.

### If multiple files
### The fingerprint_directory method three arguments:
# input directory to look for audio files
# audio extensions to look for in the input directory
# number of processes (optional)--------------------
# 
# djv.fingerprint_directory("./", [".wav"], 3)

# generate fingerprint from a single audio file
song = djv.fingerprint_file("audio.wav")